# Imports

In [ ]:
## Logging and Shell
import logging
logging.basicConfig(
    level=logging.ERROR,
    force=True
)

## Numeric imports
import pandas as pd

## File I/O
from pathlib import Path
import json

# Cheminformatics
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole

DIM    = 300
ASPECT = 3/2
IPythonConsole.molSize = (int(ASPECT*DIM), DIM)   # Change image size

# Custom imports
from polymerist.monomers import specification
from polymerist.rdutils.reactions import reactions

# Static Paths
RAW_DATA_DIR  = Path('monomer_data_raw')
FMT_DATA_DIR  = Path('monomer_data_formatted')
PROC_DATA_DIR = Path('monomer_data_processed')
RXN_FILES_DIR = Path('poly_rxns')

PDB_OUT_DIR   = Path('pdb_files')
PDB_OUT_DIR.mkdir(exist_ok=True)

MONO_INFO_DIR = Path('monomer_fragments')
MONO_INFO_DIR.mkdir(exist_ok=True)

# Load monomer and rxn data 

In [ ]:
input_data_path = PROC_DATA_DIR / '20231114_polyid_data_density_DP2-6 - 1,2 monomers_FILTERED.csv'
# input_data_path = PROC_DATA_DIR / 'nipu_urethanes_FILTERED.csv'
df = pd.read_csv(input_data_path, index_col=0)

In [ ]:
blacklisted_rxns = ['imide']#, 'vinyl']
df = df[df.mechanism.map(lambda s : s not in blacklisted_rxns)]

In [ ]:
keys = ['rxn_name']

df_grouper = df.groupby(keys)
frames = {
    mech : df_grouper.get_group(mech)
        for mech in df_grouper.groups
}

## Load pre-defined reactions with functional group and name backmap

In [ ]:
with (RXN_FILES_DIR / 'rxn_groups.json').open('r') as file: # load table of functional group for each reaction
    rxn_groups = json.load(file)

rxns = {
    rxnname : reactions.AnnotatedReaction.from_rxnfile(RXN_FILES_DIR / f'{rxnname}.rxn')
        for rxnname in rxn_groups.keys()
}

# Polymerizing fragments

## Example for polymerization of individual molecule

In [ ]:
mechname = 'polyurethane_isocyanate'
frame = frames[mechname]
row = frame.loc[frame.index[0]]
rxn_pathway = rxns[mechname]

named_reactants = {}
for i in range(2):
    reactant = Chem.MolFromSmiles(row[f'smiles_monomer_{i}'], sanitize=False)
    Chem.SanitizeMol(reactant, sanitizeOps=specification.SANITIZE_AS_KEKULE)
    display(reactant)
    named_reactants[ row[f'IUPAC_name_monomer_{i}'] ] = reactant

reactants = [i for i in named_reactants.values()]

In [ ]:
from polymerist.rdutils.amalgamation import portlib
from polymerist.rdutils.reactions import reactors

display(rxn_pathway)
reactor = reactors.PolymerizationReactor(rxn_pathway)
for (dimer, pair) in reactor.propagate(reactants):
    display(dimer)
    for frag in pair:
        display(frag)

    print('='*50)